# II - Utiliser l’API

# 3. Requêter vos premiers évènements

Requêter vos premiers évènements dans la ville de votre groupe sur la plage de
dates prédéfinies

In [ ]:
import requests
page = 1        
url = f"https://www.bandsintown.com/this-month/fetch-next/upcomingEvents?page={page}&longitude=-74.00597&latitude=40.71427&genre_query=all-genres"

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

data = requests.get(url, headers=headers).json()
data

# 4. Avoir les évènements d’une page spécifique

In [15]:
from urllib.request import urlopen, Request
from urllib import request, parse
from pprint import pprint
import json
url = f"https://www.bandsintown.com/this-month/fetch-next/upcomingEvents"
page= 1
params_dic= {
    "page" : f"{page}",
    "longitude" : -74.00597,
    "latitude" : 40.71427,
    "genre_query" : "all-genres"
}
url_param = parse.urlencode(params_dic)
full_url = f"{url}?{url_param}"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
req = Request(full_url, headers=headers)
response = urlopen(req)
content = response.read()
json_data = json.loads(content)
file_json= json.dumps(json_data, indent=4)


# 5. Identifier le maximum de page

In [ ]:
print(file_json
)
